In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time
from ComCheb import mpRemez
from ComCheb.Contours import mpHypocycloid
from mpmath import mp
mp.pretty = 1


storage = dict()

In [ ]:
## In order to update or refine reload
with open('Data/FC_Hypocycloid.pkl','rb') as fp:
    storage = pickle.load(fp)

In [ ]:
def FH_coef(n,m):
    if n ==0:
        return np.array([m])
    elif n<m:
        return np.array(n*[0]+[mp.mpf(1)])
    else:
        tmp1 = np.hstack([0,FH_coef(n-1,m)])
        tmp2 = 1/mp.mpf(m-1)*np.hstack([FH_coef(n-m,m)]+m*[0])
        return  tmp1-tmp2

In [ ]:
mp.dps = 80
n  = 10
m  = 5
R  = np.linspace(1,50,99)
FH = FH_coef(n,m)
FH = FH[FH!=0]

prec = 1e-50


t  = None
a  = None
for r in R:
    if r in storage.keys():
        t,a = storage[r][:2]
    t1 = time()
    gamma = mpHypocycloid(m=m,r=r)
    t,a,L,h,rel_err = mpRemez(gamma,n,t=t,a=a,symmetry=m,rc=1,prec=prec,Mmax=1000,reps=40,plot=0,pinfo=0,samples=2001)
    
    storage[r] = (t,a,L,h,rel_err,max(abs(FH-np.hstack([-L,1]))))
    print(r,'  \t',f'{float(storage[r][-1]):.8e}','\t',f'{rel_err:.4e}','\t',int(time()-t1))


with open('Data/FC_Hypocycloid.pkl','wb') as fp:
    pickle.dump(storage,fp)